<a href="https://colab.research.google.com/github/ks-chauhan/Kaggle-challenges/blob/main/House_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
device="cuda" if torch.cuda.is_available() else "cpu"
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)  # Prevent line breaks for wide DataFrames
from torch import nn
from timeit import default_timer
from tqdm.auto import tqdm
torch.manual_seed(42)

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving House_Price_Test.csv to House_Price_Test.csv
Saving House_Price_Train.csv to House_Price_Train.csv


In [ ]:
labelencoder=LabelEncoder()

In [ ]:
def labelencoder_fn(features,dataframe):
  for feature in features:
    dataframe[feature]=labelencoder.fit_transform(dataframe[feature])

In [ ]:
df_train=pd.read_csv("House_Price_Train.csv")

In [ ]:
df=pd.read_csv("House_Price_Train.csv")

In [ ]:
list_deletion=list(df_train.columns[df_train.isnull().any()])

In [ ]:
df_train.drop(list_deletion,axis=1,inplace=True)

In [ ]:
listed_datatypes_encoding=list(df_train.select_dtypes(include=['object']).columns)

In [ ]:
labelencoder_fn(listed_datatypes_encoding,df_train)

In [ ]:
print(df_train.isnull().sum().sum())  # Should be 0

0


In [ ]:
all_features=(list(df_train.columns))

In [ ]:
input_features=all_features[1:-1]
output_features=all_features[-1]

In [ ]:
len(input_features)

60

In [ ]:
X_train_numpy=df_train[input_features].values
y_train_numpy=df_train[output_features].values

In [ ]:
y_train_numpy

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y_scaler=StandardScaler()
X_train_standardized = scaler.fit_transform(X_train_numpy)  # Standardizing features
X_train_standardized = torch.tensor(X_train_standardized, dtype=torch.float32)  # Convert to tensor
y_train_standardized=y_scaler.fit_transform(y_train_numpy.reshape(-1,1))
y_train_standardized=torch.tensor(y_train_standardized,dtype=torch.float32)

In [ ]:
y_train_standardized

tensor([[ 0.3473],
        [ 0.0073],
        [ 0.5362],
        ...,
        [ 1.0776],
        [-0.4885],
        [-0.4208]])

In [ ]:
train_split=int(0.8*len(X_train_numpy))

In [ ]:
X_train_standardized.shape

torch.Size([1460, 60])

In [ ]:
X_train=X_train_standardized[:train_split]
y_train=y_train_standardized[:train_split]

In [ ]:
X_validation=X_train_standardized[train_split:]
y_validation=y_train_standardized[train_split:]

In [ ]:
X_validation.shape

torch.Size([292, 60])

In [ ]:
class HousePricePredictionV0Model(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_neurons:int,
               output_shape:int):
    super().__init__()
    self.layer=nn.Sequential(
        nn.Linear(in_features=input_shape,out_features=hidden_neurons),
        nn.ReLU(),
        nn.Linear(in_features=hidden_neurons,out_features=hidden_neurons),
        nn.ReLU(),
        nn.Linear(in_features=hidden_neurons,out_features=output_shape)
    )
  def forward(self,X:torch.Tensor):
    output=self.layer(X)
    return output

In [ ]:
model_0=HousePricePredictionV0Model(input_shape=60,hidden_neurons=40,output_shape=1).to(device)

In [ ]:
loss_fn=torch.nn.MSELoss()
optimizer=torch.optim.Adam(params=model_0.parameters(),
                          lr=0.0001)

In [ ]:
epochs=2000

In [ ]:
y_train_denormalized=y_scaler.inverse_transform(y_train.detach().numpy())

In [ ]:
start_time=default_timer()
for epoch in tqdm(range(epochs)):
  model_0.train()
  prediction=model_0(X_train)
  loss=loss_fn(prediction.squeeze(dim=1),y_train.squeeze(dim=1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if(epoch%400==0 or epoch == 0):
    prediction_denormalized=y_scaler.inverse_transform(prediction.detach().numpy())
    print(f"current loss = {loss} and accuracy : {MAPE_accuracy(prediction_denormalized,y_train_denormalized)}")
end_time=default_timer()
print(f"total time taken : {end_time-start_time}")

  0%|          | 0/2000 [00:00<?, ?it/s]

current loss = 0.9960790276527405 and accuracy : 59.62749481201172
current loss = 0.1150924488902092 and accuracy : 89.95669937133789
current loss = 0.07184485346078873 and accuracy : 92.04775714874268
current loss = 0.04986703395843506 and accuracy : 93.20706367492676
current loss = 0.034917425364255905 and accuracy : 94.08350133895874
total time taken : 4.916041692999897


In [ ]:
model_0.eval()
with torch.inference_mode():
  pred=model_0(X_validation)

In [ ]:
pred_denormalized=y_scaler.inverse_transform(pred.detach().numpy())
y_validation_denormalized=y_scaler.inverse_transform(y_validation.detach().numpy())

In [ ]:
pred_denormalized.shape

(292, 1)

In [ ]:
np.array(y_validation).shape

(292, 1)

In [ ]:
def MAPE_accuracy(pred,true):
  Mape_error=np.abs((true-pred)/true)*100
  accuracy=100-np.mean(Mape_error)
  return accuracy

In [ ]:
print(f"model accuracy : {MAPE_accuracy(pred_denormalized,np.array(y_validation_denormalized))}")

model accuracy : 88.79968547821045


In [ ]:
df_test=pd.read_csv("House_Price_Test.csv")

In [ ]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
test_list_deletions=list(df_test.columns[df_test.isnull().any()])

In [ ]:
extra_null_value_columns=list(set(test_list_deletions)-set(list_deletion))

In [ ]:
extra_null_value_columns_without_string=list(set(extra_null_value_columns)-set(listed_datatypes_encoding))

In [ ]:
extra_null_value_columns_with_string=list(set(extra_null_value_columns)-set(extra_null_value_columns_without_string))

In [ ]:
def replace_string_null(features,dataframe):
  for feature in features:
    dataframe[feature].fillna(dataframe[feature].mode()[0],inplace=True)

In [ ]:
replace_string_null(extra_null_value_columns_with_string,df_test)

<ipython-input-145-26d534f2fc50>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe[feature].fillna(dataframe[feature].mode()[0],inplace=True)


In [ ]:
extra_null_value_columns_with_string

['Functional',
 'Utilities',
 'SaleType',
 'Exterior2nd',
 'Exterior1st',
 'KitchenQual',
 'MSZoning']

In [ ]:
df_test[extra_null_value_columns].isnull().sum()

,0
Functional,0
BsmtUnfSF,0
Utilities,0
GarageCars,0
BsmtHalfBath,0
SaleType,0
BsmtFinSF1,0
Exterior1st,0
Exterior2nd,0
GarageArea,0


In [ ]:
df_test.drop(list_deletion,axis=1,inplace=True)

In [ ]:
df_test[extra_null_value_columns_without_string]=df_test[extra_null_value_columns_without_string].fillna(0)

In [ ]:
df_train[input_features].columns==df_test[input_features].columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
listed_datatypes_encoding

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'Functional',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
labelencoder_fn(listed_datatypes_encoding,df_test)

In [ ]:
df_train.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,8450,1,3,3,0,4,0,5,2,2,0,5,7,5,2003,2003,1,1,12,13,2,4,2,706,0,150,856,1,0,1,856,854,0,1710,1,0,2,1,3,1,2,8,6,0,2,548,2,0,61,0,0,0,0,0,2,2008,8,4,208500
1,2,20,3,9600,1,3,3,0,2,0,24,1,2,0,2,6,8,1976,1976,1,1,8,8,3,4,1,978,0,284,1262,1,0,1,1262,0,0,1262,0,1,2,0,3,1,3,6,6,1,2,460,2,298,0,0,0,0,0,0,5,2007,8,4,181500
2,3,60,3,11250,1,0,3,0,4,0,5,2,2,0,5,7,5,2001,2002,1,1,12,13,2,4,2,486,0,434,920,1,0,1,920,866,0,1786,1,0,2,1,3,1,2,6,6,1,2,608,2,0,42,0,0,0,0,0,9,2008,8,4,223500
3,4,70,3,9550,1,0,3,0,0,0,6,2,2,0,5,7,5,1915,1970,1,1,13,15,3,4,0,216,0,540,756,1,2,1,961,756,0,1717,1,0,1,0,3,1,2,7,6,1,3,642,2,0,35,272,0,0,0,0,2,2006,8,0,140000
4,5,60,3,14260,1,0,3,0,2,0,15,2,2,0,5,8,5,2000,2000,1,1,12,13,2,4,2,655,0,490,1145,1,0,1,1145,1053,0,2198,1,0,2,1,4,1,2,9,6,1,3,836,2,192,84,0,0,0,0,0,12,2008,8,4,250000


In [ ]:
df_train.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,8450,1,3,3,0,4,0,5,2,2,0,5,7,5,2003,2003,1,1,12,13,2,4,2,706,0,150,856,1,0,1,856,854,0,1710,1,0,2,1,3,1,2,8,6,0,2,548,2,0,61,0,0,0,0,0,2,2008,8,4,208500
1,2,20,3,9600,1,3,3,0,2,0,24,1,2,0,2,6,8,1976,1976,1,1,8,8,3,4,1,978,0,284,1262,1,0,1,1262,0,0,1262,0,1,2,0,3,1,3,6,6,1,2,460,2,298,0,0,0,0,0,0,5,2007,8,4,181500
2,3,60,3,11250,1,0,3,0,4,0,5,2,2,0,5,7,5,2001,2002,1,1,12,13,2,4,2,486,0,434,920,1,0,1,920,866,0,1786,1,0,2,1,3,1,2,6,6,1,2,608,2,0,42,0,0,0,0,0,9,2008,8,4,223500
3,4,70,3,9550,1,0,3,0,0,0,6,2,2,0,5,7,5,1915,1970,1,1,13,15,3,4,0,216,0,540,756,1,2,1,961,756,0,1717,1,0,1,0,3,1,2,7,6,1,3,642,2,0,35,272,0,0,0,0,2,2006,8,0,140000
4,5,60,3,14260,1,0,3,0,2,0,15,2,2,0,5,8,5,2000,2000,1,1,12,13,2,4,2,655,0,490,1145,1,0,1,1145,1053,0,2198,1,0,2,1,4,1,2,9,6,1,3,836,2,192,84,0,0,0,0,0,12,2008,8,4,250000


In [ ]:
X_test_numpy=df_test[input_features].values

In [ ]:
X_test_standardized = scaler.fit_transform(X_test_numpy)  # Standardizing features
X_test_standardized = torch.tensor(X_test_standardized, dtype=torch.float32)

In [ ]:
model_0.eval()
with torch.inference_mode():
  test_prediction=model_0(X_test_standardized)

In [ ]:
test_prediction_denormalized=y_scaler.inverse_transform(test_prediction.detach().numpy())

In [ ]:
test_prediction_denormalized.shape

(1459, 1)

In [ ]:
submission_csv=pd.DataFrame({'Id':df_test['Id'],'SalePrice' : test_prediction_denormalized.squeeze()})

In [ ]:
submission_csv.to_csv("submission_csv",index=False)

In [ ]:
submission_csv['SalePrice'].min(),submission_csv['SalePrice'].max()

(16548.5859375, 554625.375)